In [1]:
import pandas as pd
import plotly.express as px
from chdb import session as chs
from datetime import datetime, timedelta
import json

sess = chs.Session("/Users/davidefloriello/genai/chdb_data")

query = """
SELECT ts, price, ofi
FROM market_ticks
WHERE ts >= now() - INTERVAL 30 MINUTE
ORDER BY ts
"""
result = sess.query(query, "JSONEachRow")

raw = str(result)

rows = [json.loads(line) for line in raw.splitlines() if line.strip()]

df = pd.DataFrame(rows)
df["ts"] = pd.to_datetime(df["ts"])

fig = px.line(
    df,
    x="ts",
    y=["price", "ofi"],
    labels={"value": "Value", "ts": "Timestamp"},
    title="BTC/USDT – Last 30 Minutes with OFI Overlay",
)


fig.data[1].update(yaxis="y2")

fig.update_layout(
    template="plotly_dark",
    legend_title_text="Metric",
    yaxis=dict(
        title="Price",
        side="left"
    ),
    yaxis2=dict(
        title="OFI",
        overlaying="y",
        side="right",
        showgrid=False
    ),
)